# 4. Assignement - HDR, response function estimation and HDR composition by Debevec method

In [ ]:
import exif
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
from scipy.sparse.linalg import lsqr
from copy import copy
import requests
import os
import shutil
import tempfile


### Helper functions

In [ ]:
def download_images(urls, path):
    for idx in range(len(urls)):
        print(urls[idx], end='')
        r = requests.get(urls[idx], stream=True)
        if r.status_code == 200:
            with open(os.path.join(path, f'image{idx:02d}.jpg'), 'bw') as f:
                # r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)    
                print('...OK')
        else:
                print('...FAIL')

def read_images(file_pattern, scale_percent = 100):
    files = glob.glob(file_pattern) 

    imgs = []
    t = []

    for file in files:
        tmp_img = cv2.imread(file) 
        width = int(tmp_img.shape[1] * scale_percent // 100) 
        height = int(tmp_img.shape[0] * scale_percent // 100)
        dim = (width, height) 
        imgs.append(cv2.cvtColor(cv2.resize(tmp_img, dim, interpolation = cv2.INTER_AREA), cv2.COLOR_BGR2RGB)) 
        info = exif.Image(file)
        t.append(info.exposure_time) 
    return (height, width), np.array(imgs), t 

def get_weights(Z, L):
    return np.interp(Z, [0, (L-1)/2, L-1], [0, 1, 0]).flatten() 


### Download images and prepare vectors/matrices

In [ ]:
images_urls = ['https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka02.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka03.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka04.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka05.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka06.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka07.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka08.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka09.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka10.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka11.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka12.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka13.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka14.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka15.jpg?raw=true',
              ]

with tempfile.TemporaryDirectory() as tempdir:
    download_images(images_urls, tempdir)
    dim, Z, t = read_images(os.path.join(tempdir, "*.jpg"), scale_percent = 0.3)

sort_index = sorted(range(len(t)), key=lambda k: t[k])
t = [t[k] for k in sort_index]
Z = Z[sort_index]

print(f'Image shape: {dim}')
print(f'Exposure times [s]: {t}')

r = Z[:, :, :, 0].flatten() 
g = Z[:, :, :, 1].flatten() 
b = Z[:, :, :, 2].flatten() 

# weight vectors
w_r = get_weights(r, 2**8) 
w_g = get_weights(g, 2**8) 
w_b = get_weights(b, 2**8) 

# Indices to time vector and images
t_ind, Z_ind = np.indices((Z.shape[0], Z.shape[1]*Z.shape[2]))
t_ind = t_ind.flatten()
E_ind = Z_ind

### Estimate exposure times function

In [ ]:
def estimate_exposure_time(w, Z, t_ind, Z_ind):
    '''
    Estimates exposure times assumes the camera response function is identity.

    Parameters:
    w (float): An array of shape (N,) of weights.
    Z (numpy.ndarray): An array of shape (N,) containing the pixel intezities.
    t_ind (numpy.ndarray): An array of shape (P, N,) containing the time indices.
    Z_ind (numpy.ndarray): An array of shape (P, N,) containing the pixel intensities indices.

    Returns:
    numpy.ndarray: An array of shape (P,) containing the estimated exposure times.

    '''
    # Z = Z/(L-1)
    eps = np.finfo(float).eps
    a_rows_count = (np.max(Z_ind) + 1) * (np.max(t_ind) + 1) + 1
    A = np.zeros((a_rows_count, np.max(Z_ind) + np.max(t_ind) + 2))
    b = np.zeros((a_rows_count))
    Z_ind = Z_ind.flatten()
    r_ind = list(range(A.shape[0]-1))
    A[r_ind, Z_ind.flatten()] = np.sqrt(w)
    A[r_ind, t_ind.flatten() + 1 + np.max(Z_ind)] = np.sqrt(w)
    b[:-1] = np.log(Z + eps) * np.sqrt(w)
    A[-1, np.max(Z_ind) + 1] = 1
    sol = lsqr(A, b)
    return sol[0][np.max(Z_ind) + 1:]


In [ ]:
t_est_r = estimate_exposure_time(w_r, r, t_ind, E_ind)
t_est_g = estimate_exposure_time(w_g, g, t_ind, E_ind)
t_est_b = estimate_exposure_time(w_b, b, t_ind, E_ind)
print(np.log(t[0]),t_est_r)

In [ ]:
print(f'Exposure time estimation (red): ', np.exp(t_est_r))
print(f'Exposure time estimation (green): ', np.exp(t_est_g))
print(f'Exposure time estimation (blue): ', np.exp(t_est_b))
print(f'Real exposure time: ', t)

### Estimate inverse response function - YOUR TASK

In [ ]:
def estimate_response_debevec(w, L, Z, t_vec, E_ind, lambd):
    '''
    Estimates camera inverse response function values.

    Parameters:
    w (float): An array of shape (N,) of weights.
    L (int): number of pixel intezity discrete levels
    Z (numpy.ndarray): An array of shape (N,) containing the pixel intezities.
    t_ind (numpy.ndarray): An array of shape (P, N,) containing the time indices.
    Z_ind (numpy.ndarray): An array of shape (P, N,) containing the pixel intensities indices.

    Returns:
    numpy.ndarray: An array of shape (L,) containing the estimated camera inverse response function values.
    '''
    
    # YOUR estimate_response_debevec LOGIC HERE


In [ ]:
lambda_ = 2
g_r, _ = estimate_response_debevec(w_r, 2**8, r, t, Z_ind, lambda_)
g_g, _ = estimate_response_debevec(w_g, 2**8, g, t, Z_ind, lambda_)
g_b, _ = estimate_response_debevec(w_b, 2**8, b, t, Z_ind, lambda_)
plt.plot(g_r, 'r')
plt.plot(g_g, 'g')
plt.plot(g_b, 'b')
plt.title('Estimated camera inverse response functions values');
plt.xlabel('Z')
plt.ylabel('g()');

### Fit the models of inverse response functions 

In [ ]:
def fit_response(L, g, degree=2):
    g_val = []
    for idx in range(L):
        g_val.append([idx, g[idx]])
    g_val = np.array(g_val)
    
    poly_transform = PolynomialFeatures(degree=degree, include_bias=False).fit(g_val[:,0].reshape((-1,1)))
    poly_trans = lambda x: poly_transform.transform(x.reshape((-1,1)))
    z_poly = poly_trans(g_val[:,0])
    g_model = LinearRegression()
    g_model.fit(z_poly, g_val[:,1])
    g_model_pred = g_model.predict(poly_trans(g_val[:,0]))

    plt.figure()
    plt.scatter(x=g_val[:,0], y=g_val[:,1])
    plt.plot(g_model_pred, 'r')
    
    return g_model_pred

g_model_r = fit_response(2**8, g_r, degree=2)
plt.title('Camera response function - red chanel')
g_model_g = fit_response(2**8, g_g, degree=2)
plt.title('Camera response function - green chanel')
g_model_b = fit_response(2**8, g_b, degree=2)
plt.title('Camera response function - blue chanel');

### HDR Composition - download full scale images

In [ ]:
with tempfile.TemporaryDirectory() as tempdir:
    download_images(images_urls, tempdir)
    dim_f, Z_f, t_f = read_images(os.path.join(tempdir, "*.jpg"), scale_percent = 100)

### Convert pixel intesities to radiances - YOUR TASK

In [ ]:
def hdr_from_exposure(g, w, Z, t):
    w_img = np.sqrt(w.reshape((len(t),-1)))
    Z_img = np.interp(Z, list(range(2**8)), g).reshape((len(t),-1))
    E = np.zeros(w_img.shape[1], dtype=np.float32)
    t = np.log(t)
    
    # YOUR pixel irradiance LOGIC HERE, don't forget return E instead of log(E)
    
    return E

r_f = Z_f[:, :, :, 0].flatten() 
g_f = Z_f[:, :, :, 1].flatten() 
b_f = Z_f[:, :, :, 2].flatten() 
w_r_f = get_weights(r_f, L) 
w_g_f = get_weights(g_f, L) 
w_b_f = get_weights(b_f, L) 

E_r_est = hdr_from_exposure(g_model_r, w_r_f, r_f, t)
E_g_est = hdr_from_exposure(g_model_g, w_g_f, g_f, t)
E_b_est = hdr_from_exposure(g_model_b, w_b_f, b_f, t)


### Show radiance maps

In [ ]:
plt.imshow(np.log(E_r_est.reshape(dim_f)))
plt.colorbar()
plt.title('Radiance map - red chanel');

In [ ]:
plt.imshow(np.log(E_g_est.reshape(dim_f)))
plt.colorbar()
plt.title('Radiance map - green chanel');

In [ ]:
plt.imshow(np.log(E_b_est.reshape(dim_f)))
plt.colorbar()
plt.title('Radiance map - blue chanel');

### Show HDR

In [ ]:
hdr = np.stack((E_r_est.reshape(dim_f), E_g_est.reshape(dim_f), E_b_est.reshape(dim_f)), axis=-1)
plt.imshow(hdr)
plt.title('HDR Image - clipped');

### Convert HDR to LDR by linear tonemapping

In [ ]:
tonemap1 = cv2.createTonemap(gamma=1)
res_debevec = tonemap1.process(np.log(hdr.copy()).astype(np.float32)
res_debevec = np.clip(res_debevec*255, 0, 255).astype('uint8')
plt.imshow(res_debevec)
plt.title('Generated LDR image by linear tonemapping');

### Convert HDR to LDR by logarithmic tonemapping

In [ ]:
def hdr_to_ldr_operator(radiance_map, b=0.5, ldmax=100):
    max_radiance = np.max(radiance_map)
    a = ldmax*0.01/np.log10(max_radiance+1)
    c = np.log(b)/np.log(0.5)
    ldr = a*np.log(radiance_map+1)/(np.log(2+(radiance_map/max_radiance)**c*8))
    return ldr

def log_tonemap(hdr, b=0.5, ldmax=100):
    ldr = np.zeros(hdr.shape)
    for idx in range(hdr.shape[-1]):
        ldr[:,:,idx] = hdr_to_ldr_operator(hdr[:,:,idx], b, ldmax)
    return np.clip(ldr*255, 0, 255).astype('uint8')

ldr_debevec = log_tonemap(hdr_debevec, b=0.5, ldmax=70)
plt.imshow(cv2.cvtColor(ldr_debevec, cv2.COLOR_BGR2RGB))